In [1]:
from dotenv import load_dotenv
import os
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

In [2]:
load_dotenv()

True

In [3]:
os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [4]:
index_name = "docs-quickstart-index"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=2,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 


chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)
chat.openai_api_key = os.getenv("OPENAI_API_KEY")

result = chat.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        )
    ]
)
print(result.content)

chat.openai_api_key = os.getenv("OPENAI_API_KEY")